In [1]:
import pandas as pd
import numpy as np
import spacy
import re
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import drive
drive.mount('/content/drive')
import warnings; warnings.simplefilter('ignore')

Mounted at /content/drive


In [2]:
# global variables

# path to dataset
path = '/content/drive/MyDrive/netflix_titles(1).csv'


# create empty dictionary to store predicted data types
predicted_data_types = {}

# type map for nlp predictions
type_map = {
            'org': ['name', 'brand', 'company', 'organization', 'firm', 'hotel', 'industry'],
            'person': ['name', 'relation', 'first name', 'last name', 'middle name'],
            'gpe': ['place', 'destination', 'origin', 'landmark', 'address', 'country', 'city', 'state'],
            'fac': ['building', 'airport', 'highway', 'bridge', 'station', 'metro', 'vehicle', 'hospital'],
            'norp': ['nationality', 'religion', 'political party', 'caste', 'creed', 'race', 'group', 'congregation'],
            'loc': ['location', 'place', 'mountain range', 'water body', 'coordinates'],
            'product': ['item', 'commodity', 'product', 'machine', 'technology', 'service', 'transportation', 'book', 'investment', 'insurance', 'description'],
            'event': ['sport', 'war', 'hurricane', 'tornado', 'attack', 'tsunami', 'calamity'],
            'work_of_art': ['movie', 'book', 'music', 'song', 'art', 'pop', 'culture', 'band'],
            'time': ['arrival', 'departure', 'time zone', 'standard time', 'expiration date', 'manufacture date', 'last seen', 'occurance', 'timing'],
            'date': ['ID','serial number','identification number', 'birthday', 'anniversary', 'expiration date', 'manufacture date'],
            'percent': ['percentage', 'ratio', 'range', 'factor', 'length', 'breadth', 'height', 'altitude', 'mean sea level', 'mean', 'median', 'sensex', 'statistics', 'nifty', 'rating', 'cross-section', 'dose', 'weight', 'number', 'enumeration'],
            'money': ['unit', 'worth', 'value', 'price', 'cost', 'expense', 'currency', 'quantity', 'amount'],
            'amount': ['percentage', 'ratio', 'range', 'factor', 'length', 'breadth', 'height', 'altitude', 'mean sea level', 'mean', 'median', 'sensex', 'statistics', 'nifty', 'rating', 'cross-section', 'dose', 'weight', 'number', 'enumeration'],
            'currency': ['unit', 'worth', 'value', 'price', 'cost', 'expense', 'currency', 'quantity', 'amount', 'gain', 'profit', 'loss', 'incomme', 'salary', 'revenue', 'net worth', 'net profit', 'net loss', 'capitol', 'debit', 'credit'],
            'measurement': ['unit', 'measurement', 'length', 'breadth', 'height', 'width', 'cross-section', 'quantity', 'amount', 'number of', 'enumeration', 'debit', 'credit'],
            'cardinal': ['ID', 'identification number', 'number plate', 'latitiude', 'longitude', 'coordinates', 'paper id', 'vehicle number', 'code', 'zipcode', 'contact number', 'shipping details', 'passcode', 'password', 'identity', 'email id', 'username', 'medicine', 'url'],
            'ordinal' : ['rating', 'numerical indicator', 'size', 'level', 'status'],
            'flag' : ['Yes','No','Male','Female', '0','1','Y','N','F','M']
}



In [3]:
def classify_column_datatype(df):    
    # iterate through each column in the dataframe
    for column in df.columns:
        # check data type of column
        dtype = str(df[column].dtype)

        # predict data type based on dtype
        if dtype == 'object':
            if df[column].str.match(r'^\d+$').all():
                predicted_data_types[column] = 'numerical integer'

            elif df[column].str.match(r'^\d+\.\d+$').all():
                predicted_data_types[column] = 'numerical float'

            elif df[column].str.match(r'\b\d{2}/\d{2}/\d{4}\b').all():
                predicted_data_types[column] = 'datetime'
            
            elif df[column].str.match(r'\b\d{1,2}-\d{2}-\d{4}\b').all() or pd.to_datetime(df[column], errors='coerce').notnull().all():
                predicted_data_types[column] = 'datetime'

            elif df[column].str.match(r'\b\d{4}-\d{2}-\d{2}\b').all() or pd.to_datetime(df[column], errors='coerce').notnull().all():
                predicted_data_types[column] = 'datetime'

            elif df[column].str. match(r'\b\d{4}/\d{2}/\d{2}\b').all() or pd.to_datetime(df[column], errors='coerce').notnull().all():
                predicted_data_types[column] = 'datetime'

            elif df[column].str.match(r'\b\d{1,2}:\d{2}\b').all() or pd.to_datetime(df[column], errors='coerce').notnull().all():
                predicted_data_types[column] = 'datetime'

            elif df[column].str.match(r'\b\d{1,2} (am|pm)\b').all() or pd.to_datetime(df[column], errors='coerce').notnull().all():
                predicted_data_types[column] = 'time'    
            else:
                predicted_data_types[column] = 'categorical'
        elif dtype == 'int64':
            if df[column].isnull().any():
                predicted_data_types[column] = 'float'
            else:
                predicted_data_types[column] = 'int'
        elif dtype == 'float64':
            if df[column].isnull().any():
                predicted_data_types[column] = 'float'
            elif df[column].apply(lambda x: x.is_integer()).all():
                predicted_data_types[column] = 'int'
            else:
                predicted_data_types[column] = 'float'
        elif dtype == 'bool':
            predicted_data_types[column] = 'bool'
        else:
            predicted_data_types[column] = 'unknown'

    return predicted_data_types
    # print predicted data types
    for column, data_type in predicted_data_types.items():
        print(f"{column}: {data_type}")

In [4]:
def preprocess_df(path):
  
  read1 = pd.read_csv(path , on_bad_lines='skip')
  read1.shape
  read1.isnull()
  read1.isnull().sum()
  df_var=read1.fillna(" ")
  df_var.isnull().sum()
  predicted_data_types = classify_column_datatype(df_var)
  # df_var.head()
  return df_var

In [5]:
df1 = preprocess_df('/content/drive/MyDrive/netflix_titles(1).csv')
df1

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,,,,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,,,,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [6]:

# print predicted data types
for column, data_type in predicted_data_types.items():
  print(f"{column}: {data_type}")

show_id: categorical
type: categorical
title: categorical
director: categorical
cast: categorical
country: categorical
date_added: categorical
release_year: int
rating: categorical
duration: categorical
listed_in: categorical
description: categorical


In [7]:
#NER on csv fILE applies on every value in a column and returns the column
nlp = spacy.load("en_core_web_sm")

def get_entities(text):
    doc = nlp(text)
    return [(ent.label_) for ent in doc.ents]

# this function assigns ner value to new copy of df and returns the new df
# Extracts majority of each column and stores in an array

def assign_column_ner_classification(df,num_samples=25):
  df = df.sample(n=num_samples)
  col_types_df = pd.DataFrame(df)
  for colname in df.columns:
    col_types_df[colname] = df[colname].astype(str).apply(get_entities)

  return col_types_df

def major_col_pred_array(df,num_samples=25):
    majority_arr = []
    new_df = pd.DataFrame()
    new_df = assign_column_ner_classification(df,num_samples)
    majority_arr = np.empty(new_df.shape[1], dtype='object')


    for i, col in enumerate(new_df.columns):
    
      counts = new_df[col].value_counts()
      
      
      if counts.empty:
        majority_arr[i] = None
      else:
        
        majority_arr[i] = counts.index[0]
      
      
      if majority_arr[i]:
        print(f"{col}: {majority_arr[i]}")
      else:
        print(f"{col}: None")
    
    return majority_arr


In [8]:
assign_column_ner_classification(df1,25)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
1411,[],[],[],[PERSON],"[PERSON, WORK_OF_ART, PERSON, PERSON]",[GPE],[DATE],[DATE],[],[CARDINAL],[],"[NORP, NORP, PERSON]"
3231,[],[],[],[],"[PERSON, PERSON, PERSON, PERSON, PERSON, PERSO...",[],[DATE],[DATE],[],[CARDINAL],[],"[GPE, ORG]"
7098,[],[],[PERSON],[],"[PERSON, PERSON, PERSON, PERSON, FAC, PERSON, ...","[GPE, GPE, GPE, GPE, GPE]",[DATE],[DATE],[],[DATE],[],[PERSON]
4625,[],[],[CARDINAL],"[PERSON, PERSON]",[],[GPE],[DATE],[DATE],[],[CARDINAL],[],"[GPE, GPE]"
2517,[],[],[],[],"[PERSON, PERSON, PERSON, PERSON, PERSON, PERSO...",[GPE],[DATE],[DATE],[],[],"[ORG, ORG]",[]
4930,[],[],[ORG],[],"[PERSON, PERSON, PERSON, PERSON, PERSON, PERSO...",[GPE],[DATE],[DATE],[],[DATE],[],"[PERSON, PERSON, FAC]"
285,[],[],[],[PERSON],"[PERSON, PERSON, PERSON, PERSON, PERSON, PERSO...","[GPE, GPE, GPE]","[DATE, DATE]",[DATE],[],[CARDINAL],[],"[DATE, EVENT]"
4540,[],[],"[PERSON, PERSON]","[PERSON, PERSON]","[PERSON, PERSON, PERSON, PERSON, PERSON, PERSO...",[GPE],[DATE],[DATE],[],[CARDINAL],[ORG],"[PERSON, GPE]"
8377,[],[],[],[PERSON],"[PERSON, PERSON, PERSON, PERSON, PERSON, PERSO...","[GPE, GPE, GPE]",[DATE],[DATE],[],[CARDINAL],[],"[GPE, PERSON, PERSON]"
3357,[],[],[],[],"[ORG, PERSON, ORG, ORG, PERSON, PERSON, PERSON]",[],[DATE],[DATE],[],[],"[ORG, ORG]",[CARDINAL]


In [9]:
# df storing the majority NER detected for each column, if not detected then column value = None
new_maj_col = major_col_pred_array(df1,25)

show_id: None
type: None
title: None
director: ['PERSON']
cast: None
country: ['GPE']
date_added: ['DATE']
release_year: ['DATE']
rating: None
duration: ['CARDINAL']
listed_in: None
description: None


In [11]:
# for i in new_maj_col:
#   print(i)


# new_maj_col.tolist()[2][0]

In [12]:
columns = new_maj_col.tolist()
for i in range(12):
  columns[i]

In [13]:
## need to make changes in this file for integration
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def match_column_type(dataset, column_name):
    # Create a TfidfVectorizer object
    vectorizer = TfidfVectorizer()
    
    # Fit the vectorizer on the text data
    vectorizer.fit(dataset[column_name])
    
    # Create a dictionary to store the similarity scores
    similarity_scores = {}
    
    # Iterate over the type map
    for key, values in type_map.items():
        # Create a string by joining the values
        values_str = ' '.join(values)
        
        # Transform the values string and the text data into vectors
        values_vec = vectorizer.transform([values_str])
        text_vec = vectorizer.transform(dataset[column_name])
        
        # Compute the cosine similarity between the values vector and the text vectors
        similarity = cosine_similarity(values_vec, text_vec)
        
        # Compute the average similarity score
        avg_similarity = similarity.mean()
        
        # Store the average similarity score in the dictionary
        similarity_scores[key] = avg_similarity
    
    # Find the key with the highest similarity score
    best_match = max(similarity_scores, key=similarity_scores.get)
    
    return best_match


import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import spacy

nlp = spacy.load("en_core_web_sm")

def predict_matching_values(df, column_name, key):
    
    num_columns = df.shape[1]
    num_columns=num_columns


    columns = new_maj_col.tolist()
    

    # Get the embeddings for the column name and key
    column_emb = nlp(str(column_name).lower()).vector.reshape(1, -1)
    key_emb = nlp(key).vector.reshape(1, -1)

    # Compute the cosine similarity between the column name and key embeddings
    sim = cosine_similarity(column_emb, key_emb)

    # Get the most similar value from the type map for the given key
    key_values = type_map[key]
    most_similar_value = max(key_values, key=lambda x: cosine_similarity(nlp(x).vector.reshape(1, -1), key_emb))

    # Create a list of values that closely match the column name
    matching_values = []
    for value in key_values:
        value_emb = nlp(value).vector.reshape(1, -1)
        value_sim = cosine_similarity(column_emb, value_emb)
        if value_sim >= sim:
          matching_values.append(value)

    return matching_values


def detect1_semantic_type(values):
    """Detects the semantic type of a list of numeric values."""
    
    # Initialize variables
    is_continuous = True
    is_discrete = True
    is_latitude = False
    is_longitude = False
    is_temperature = False
    is_idno = False
    is_altitude = False
    is_cost = False
    is_measurement = False
    is_salary = False
    
    # Calculate range and differences
    value_range = max(values) - min(values)
    value_diff = [values[i+1] - values[i] for i in range(len(values)-1)]
    diff_range = max(value_diff) - min(value_diff)
    # print(value_diff)
    # Check for discrete or continuous values
    if diff_range != 0:
        is_discrete = False
    else:
        is_continuous = False
    
    # # Check for latitude or longitude values
    if all(value >= -90 and value <= 90 and value > 10 for value in values):
        is_latitude = True
    elif all(value >= -180 and value <= 180 and value > 10 for value in values):
        is_longitude = True
        
    # Check for temperature values
    if value_range > 50 and all(value >= -10 and value <=50 for value in values):
        if all(abs(value) <= 10 for value in value_diff):
            is_temperature = True
        
    # Check for values
    if value_range <= 100 and all(value >= 0 and value <= 100 for value in values):
        is_idno = True
        
    # Check for altitude values
    if value_range > 1000 and all(value >= -413.6 and value <= 8848  for value in values):
        is_altitude = True
    
    # Check for cost values
    if value_range > 100 and any(value < 0 for value in values):
        is_cost = True
        
    # Check for measurement values
    if value_range > 1000 and any(value < 0 for value in values):
        is_measurement = True
        
    # Check for salary values
    if value_range > 1000 and all(value >= 0 for value in values):
        is_salary = True
        
    # Return results
    if is_latitude:
        return "Latitude"
    elif is_longitude:
        return "Longitude"
    elif is_temperature:
        return "Temperature"
    elif is_idno:
        return "ID number"
    elif is_altitude:
        return "Numeric Indicators"
    elif is_cost:
        return "Cost"
    elif is_measurement:
        return "Measurement"
    elif is_salary:
        return "Count"


def classify_string_column_three(col_text):
    # Define regex patterns for different entities
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    phone_pattern = r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b'
    pincode_pattern = r'\b\d{6}\b'
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    # Check if column matches any of the patterns
    if re.search(email_pattern, col_text):
        return 'email'
    elif re.search(phone_pattern, col_text):
        return 'phone number'
    elif re.search(pincode_pattern, col_text):
        return 'pincode'
    elif re.search(url_pattern, col_text):
        return 'url'
    else:
        return None


def classify_columns(data, batch_size=1000):
    data = data.iloc[:30,:]
    """
    Classifies columns in a CSV dataset using NER and regex-based entity classification.

    :param data: A Pandas DataFrame containing the CSV dataset.
    :param batch_size: The size of the batches to use for processing the dataset.
    :return: A Pandas DataFrame with column names and their predicted entity types.
    """
    predictions = []

    str_cols = [col for col in predicted_data_types if 'categorical' in predicted_data_types[col]]
    numeric_cols = [col for col in predicted_data_types if 'int' in predicted_data_types[col] or 'numerical' in predicted_data_types[col]]
    print()
    print("printing the columns : ")
    print("str: ",str_cols)
    print("numeric : ",numeric_cols)
    print()
    # Classify object type columns using NLP and regex

    i = 0
    for col in str_cols:
        flag_tm = 0
        # If no named entity is found, use regex to classify the column
        col_type = None
        col_text = ' '.join(data[col].fillna('').values)
        col_type = classify_string_column_three(col_text)


        if col_type:
          predictions.append({'colname': col, 'prediction' : col_type})


        if not col_type:
            num_columns = data.shape[1]
            columns = []
            columns = new_maj_col.tolist()
            
            i = data.columns.get_loc(col)
            if columns[i]:
              match_value = new_maj_col.tolist()[i][0]
              col_type =  match_value
              #col_type = predict_matching_values(data, col, columns[i], columns)
              col_vals = [word for word in type_map[match_value.lower()]]
              predictions.append({'colname': col, 'prediction': col_vals})
              #print("based on column ner type = ",col, col_type, col_vals)
              
            else:    
              col_text_word = col_text.split()[0]
              # print(col_text_word)
              if re.search(r"[MF]", col_text_word, re.IGNORECASE):
                  col_type = 'Gender'
                  predictions.append({'colname': col, 'prediction': col_type})
              elif re.search(r"^[a-zA-Z]$", col_text_word, re.IGNORECASE):
                  col_type = 'Class'
                  predictions.append({'colname': col, 'prediction': col_type})
              elif re.search(r"^\d$", col_text_word, re.IGNORECASE):
                  col_type = 'Number|Count'
                  predictions.append({'colname': col, 'prediction': col_type})
              else:
                  key = new_maj_col.tolist()[df1.columns.get_loc(col)]
                  if key:
                    col_type = predict_matching_values(data, col, key[0])
                    predictions.append({'colname': col, 'prediction': col_type})

                  else:
                    predictions.append({'colname': col, 'prediction': key})
                                     
    # Classify numerical columns using range-based hierarchical classification
    for col in numeric_cols:
        if detect1_semantic_type(data[col]):
          col_type = detect1_semantic_type(data[col])
          predictions.append({'colname': col, 'prediction': col_type})
                     
        else:
            col_range = data[col].max() - data[col].min()
          
            if col_range == 0:
                col_type = 'constant'
            elif col_range == 1:
                col_type = 'binary'
            elif col_range <= 10:
                col_type = 'ordinal'
            elif col_range <= 100:
                col_type = 'interval'
            else:
                col_type = 'ratio'
            predictions.append({'colname': col, 'prediction': col_type})
                               
    return pd.DataFrame(predictions)

In [14]:
classify_columns(df1)


printing the columns : 
str:  ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'rating', 'duration', 'listed_in', 'description']
numeric :  ['release_year']



,colname,prediction
0,show_id,[]
1,type,Gender
2,title,[]
3,director,"[name, relation, first name, last name, middle..."
4,cast,Gender
5,country,"[place, destination, origin, landmark, address..."
6,date_added,"[ID, serial number, identification number, bir..."
7,rating,[]
8,duration,"[ID, identification number, number plate, lati..."
9,listed_in,Gender


In [15]:
#for col in predicted_data_types:
#  print(new_maj_col.tolist()[df1.columns.get_loc(col)])